In [10]:
!pip3 install pickle5
import pickle5 as pickle
from keras_preprocessing.text import tokenizer_from_json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
#loading tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)


In [12]:
# loading our model
from keras.models import load_model
model = load_model('skill_prediction.h5')

In [13]:
from keras_preprocessing.sequence import pad_sequences
def predict(model,tokenizer,word):
    input = pad_sequences(tokenizer.texts_to_sequences([word]),maxlen=20)
    # input = tokenizer.texts_to_sequences([word])
    d = model.predict(input)
    # print('probebilty ',d)
    if d <= 0.5:
        d = 0 #'non skill word'
    else:
        d = 1 #'skill word' 
    return d

In [36]:
import pandas as pd
import numpy as np
DA = pd.read_csv('dataset.csv')
DA

,Title,Description
0,Data Analyst,Job Summary:\nThe Data Analyst will be respons...
1,Data Analyst,Who we are\nMoneris was created as a joint inv...
2,Data Analyst,EXTERNAL JOB POSTING #124\nPosted: 04 October ...
3,Data Analyst,We are looking for someone that can follow ins...
4,Data Analyst,The Data Analyst will be responsible for provi...
...,...,...
70,machine learning engineer,Insights yielded by this patented software tec...
71,machine learning engineer,Machine Learning Engineer\nPosition: Full time...
72,machine learning engineer,This is an opportunity to take your machine le...
73,machine learning engineer,"As a Machine Learning Engineer, you'll help de..."


In [37]:
desc = DA.Description

In [38]:
desc

0     Job Summary:\nThe Data Analyst will be respons...
1     Who we are\nMoneris was created as a joint inv...
2     EXTERNAL JOB POSTING #124\nPosted: 04 October ...
3     We are looking for someone that can follow ins...
4     The Data Analyst will be responsible for provi...
                            ...                        
70    Insights yielded by this patented software tec...
71    Machine Learning Engineer\nPosition: Full time...
72    This is an opportunity to take your machine le...
73    As a Machine Learning Engineer, you'll help de...
74    Successful candidates will demonstrate excelle...
Name: Description, Length: 75, dtype: object

In [17]:
import spacy
#Initializing the spacy engine and then use it to process the text:
nlp = spacy.load('en_core_web_sm')



In [25]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
stop_words = set(stopwords.words("english"))
# remove stopwords function
def remove_stopwords(text):
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    r = TreebankWordDetokenizer().detokenize(filtered_text)
    return r
    
# example_text = ["This is a sample", "sentence and we are", "going to remove" ,"the stopwords from this."]
# for i in example_text:
#     remove_stopwords(i)

In [21]:
import nltk
import string
import re

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

def text_preprocess(text):
    r = text.lower() # coverting into lower case
    r = remove_stopwords(r) # removing stop words
    r = re.sub(r'\d+', '', r) # removing numbers
    r = remove_punctuation(r) # removing punctuations
    r = " ".join(r.split()) # removing white sapce
    r = ''.join(r.splitlines()) # removing newline character
    return r
    




In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [39]:
df_nouns = np.empty(shape=(len(desc)),dtype=object)
for i in range(len(desc)):
    nouns = []
    doc = nlp(str(desc[i]).lower())
    for chunk in doc.noun_chunks:
        chunk = text_preprocess(str(chunk))
        nouns.append(chunk)
    df_nouns[i] = nouns


In [40]:
df_nouns.shape

(75,)

In [41]:
type(df_nouns)

numpy.ndarray

In [42]:

df_nouns[0][0]

'job summary'

In [32]:
df_nouns

array([list(['job summary', 'data analyst', 'data', 'needs', 'brokerage', 'role', 'internal stakeholders', 'business problem formulation', 'relevant data sources', 'sub', '', 'processes', '', 'new kpis', 'new data extract requirements', 'data', 'multiple sources', 'highquality insights', '', 'full narrative', 'role', 'significant amount', 'collaboration', 'successful person', 'multiple critical projects', '', 'diverse inclusive authentic workplace', 'position overview', 'advanced skills', 'data analysis', 'profiling', 'data mapping', 'data modeling', 'data lakes', 'analytics data analytics', 'aws quicksight redshift data migration', 'sql', 'etl scripting', 'integration', 'rest apis', 'nodejs aws lambdaapi gateway detailed duties', 'scriptingprogramming skills', 'various types', 'source data', 'wellprepared datasets', 'multiple levels', 'granularities', 'eg demographics', 'customers', 'products', 'transactions', 'development', 'standard customized reporting', 'dashboards', 'analysis', '

In [ ]:
df_copy =  df_nouns.copy()

In [33]:
from keras_preprocessing.sequence import pad_sequences
def predict(model,tokenizer,word):
    input = pad_sequences(tokenizer.texts_to_sequences([word]),maxlen=20)
    # input = tokenizer.texts_to_sequences([word])
    d = model.predict(input)
    # print(d)
    # print('probebilty ',d)
    if d <= 0.5:
        d = 0 #'non skill word'
    else:
        d = 1 #'skill word' 
    # print(d)
    return d

In [43]:
len(df_nouns)

75

In [ ]:
for j in df_nouns[1]:
  print(j)

In [52]:
dir_noun = dict()
# dir_noun[df_nouns[0][0]] = 1
for i in range(len(df_nouns)):
    dir_temp = dict()
    for j in df_nouns[i]:
        # print(type(i))
        r = predict(model,tokenizer,j)
        if r == 1:
          dir_temp[j] = r
    dir_noun[i] = dir_temp


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 30ms/step


In [53]:
len(dir_noun)

75

In [55]:
import json
with open("dir_json.json", "w") as outfile:
    json.dump(dir_noun, outfile)

In [54]:
dir_noun

{0: {'data analyst': 1,
  'data': 1,
  'new data extract requirements': 1,
  'advanced skills': 1,
  'data analysis': 1,
  'data mapping': 1,
  'data modeling': 1,
  'data lakes': 1,
  'analytics data analytics': 1,
  'aws quicksight redshift data migration': 1,
  'sql': 1,
  'etl scripting': 1,
  'integration': 1,
  'rest apis': 1,
  'nodejs aws lambdaapi gateway detailed duties': 1,
  'source data': 1,
  'wellprepared datasets': 1,
  'development': 1,
  'dashboards': 1,
  'analysis': 1,
  'statistical models': 1,
  'handson development': 1,
  'analytical solutions': 1,
  'available data': 1,
  'analytical needs': 1,
  'statistical findings': 1,
  'appropriate algorithms': 1,
  'data structures': 1,
  'analyst': 1,
  'programming languages': 1,
  'graphql': 1,
  'nodejs aws lambdaapi gateway experience': 1,
  'data mining': 1,
  'quantitative analysis experience': 1,
  'machine learning algorithms': 1,
  'associated data': 1,
  'analytical problem solving datadrivenmetrics driven job 

In [56]:
final_dataset = []
for i in dir_noun:
    temp_data = []
    for j in dir_noun[i]:
        temp_data.append(j)
    final_dataset.append(temp_data)

In [58]:
len(final_dataset)

75

In [59]:
final_dataset

[['data analyst',
  'data',
  'new data extract requirements',
  'advanced skills',
  'data analysis',
  'data mapping',
  'data modeling',
  'data lakes',
  'analytics data analytics',
  'aws quicksight redshift data migration',
  'sql',
  'etl scripting',
  'integration',
  'rest apis',
  'nodejs aws lambdaapi gateway detailed duties',
  'source data',
  'wellprepared datasets',
  'development',
  'dashboards',
  'analysis',
  'statistical models',
  'handson development',
  'analytical solutions',
  'available data',
  'analytical needs',
  'statistical findings',
  'appropriate algorithms',
  'data structures',
  'analyst',
  'programming languages',
  'graphql',
  'nodejs aws lambdaapi gateway experience',
  'data mining',
  'quantitative analysis experience',
  'machine learning algorithms',
  'associated data',
  'analytical problem solving datadrivenmetrics driven job type'],
 ['models',
  'business data requirements',
  'complete reporting analytics solutions',
  'extensive sq

In [64]:

df_data = pd.DataFrame(zip(final_dataset),columns=['keywords'])

In [73]:
pd.DataFrame(final_dataset).to_csv('dataset_1.csv')

In [65]:
df_data

,keywords
0,"[data analyst, data, new data extract requirem..."
1,"[models, business data requirements, complete ..."
2,"[data analyst, data collection, analysis, cota..."
3,"[microsoft excel, jr research analyst, reporti..."
4,"[data analyst, interpreting, data, data analys..."
...,...
70,"[artificial intelligence, algorithms, producti..."
71,"[personal professional development, developmen..."
72,"[machine learning, ai, data, machine intellige..."
73,"[machine learning engineer, deep learning mode..."


In [69]:
df = [DA['Title'],df_data]
keyword_data = pd.concat(df,axis=1)

In [70]:
keyword_data

,Title,keywords
0,Data Analyst,"[data analyst, data, new data extract requirem..."
1,Data Analyst,"[models, business data requirements, complete ..."
2,Data Analyst,"[data analyst, data collection, analysis, cota..."
3,Data Analyst,"[microsoft excel, jr research analyst, reporti..."
4,Data Analyst,"[data analyst, interpreting, data, data analys..."
...,...,...
70,machine learning engineer,"[artificial intelligence, algorithms, producti..."
71,machine learning engineer,"[personal professional development, developmen..."
72,machine learning engineer,"[machine learning, ai, data, machine intellige..."
73,machine learning engineer,"[machine learning engineer, deep learning mode..."


In [71]:
keyword_data.to_csv('keyword_datset')

In [ ]:
type(df_nouns[0])

In [ ]:
dir_noun

In [ ]:
dir_noun = {}
for i in df_nouns:
    

In [ ]:
df_nouns[0][0]

In [ ]:
df_copy[0][0]

In [ ]:
for i in df_nouns[0]:
    predict(model,tokenizer,i)